This is an example to use python and beautifulsoup to web scrapping weather data from website https://forecast.weather.gov/MapClick.php?lat=44.9207&lon=-92.9632#.YNjE0ehKiHs

In [4]:
# Import Request and Beautifulsoup packages

import requests
from bs4 import BeautifulSoup

# Using request.get to get the content from html
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=44.9207&lon=-92.9632#.YNjE0ehKiHs")

# Create an instance of BS class to parese the content
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  This
  <br/>
  Afternoon
 </p>
 <p>
  <img alt="This Afternoon: A 40 percent chance of showers and thunderstorms.  Mostly sunny, with a high near 79. West southwest wind around 5 mph. " class="forecast-icon" src="newimages/medium/hi_tsra40.png" title="This Afternoon: A 40 percent chance of showers and thunderstorms.  Mostly sunny, with a high near 79. West southwest wind around 5 mph. "/>
 </p>
 <p class="short-desc">
  Chance
  <br/>
  T-storms
 </p>
 <p class="temp temp-high">
  High: 79 °F
 </p>
</div>


In [5]:
# Extract the info from the page
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()
print(period)
print(short_desc)
print(temp)

ThisAfternoon
ChanceT-storms
High: 79 °F


In [6]:
img = tonight.find("img")
desc = img['title']
print(desc)

This Afternoon: A 40 percent chance of showers and thunderstorms.  Mostly sunny, with a high near 79. West southwest wind around 5 mph. 


In [7]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['ThisAfternoon',
 'Tonight',
 'Monday',
 'MondayNight',
 'Tuesday',
 'TuesdayNight',
 'Wednesday',
 'WednesdayNight',
 'Thursday']

In [9]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]
print(short_descs)
print(temps)
print(descs)

['ChanceT-storms', 'ChanceT-storms thenPartly Cloudy', 'Mostly Sunnythen ChanceT-storms', 'ChanceT-storms', 'ChanceT-storms', 'Slight ChanceT-storms', 'Sunny thenSlight ChanceT-storms', 'Slight ChanceT-storms', 'Mostly Sunnythen SlightChanceT-storms']
['High: 79 °F', 'Low: 65 °F', 'High: 83 °F', 'Low: 66 °F', 'High: 83 °F', 'Low: 65 °F', 'High: 86 °F', 'Low: 64 °F', 'High: 80 °F']
['This Afternoon: A 40 percent chance of showers and thunderstorms.  Mostly sunny, with a high near 79. West southwest wind around 5 mph. ', 'Tonight: A 30 percent chance of showers and thunderstorms, mainly before 8pm.  Partly cloudy, with a low around 65. South southwest wind around 5 mph becoming calm  in the evening. ', 'Monday: A 50 percent chance of showers and thunderstorms, mainly after 3pm.  Mostly sunny, with a high near 83. Calm wind becoming south around 5 mph.  New rainfall amounts of less than a tenth of an inch, except higher amounts possible in thunderstorms. ', 'Monday Night: A 40 percent cha

In [10]:
# Combining all the data into a Pandas Dataframe

import pandas as pd
weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc":descs
})
weather

,period,short_desc,temp,desc
0,ThisAfternoon,ChanceT-storms,High: 79 °F,This Afternoon: A 40 percent chance of showers...
1,Tonight,ChanceT-storms thenPartly Cloudy,Low: 65 °F,Tonight: A 30 percent chance of showers and th...
2,Monday,Mostly Sunnythen ChanceT-storms,High: 83 °F,Monday: A 50 percent chance of showers and thu...
3,MondayNight,ChanceT-storms,Low: 66 °F,Monday Night: A 40 percent chance of showers a...
4,Tuesday,ChanceT-storms,High: 83 °F,Tuesday: A 50 percent chance of showers and th...
5,TuesdayNight,Slight ChanceT-storms,Low: 65 °F,Tuesday Night: A 20 percent chance of showers ...
6,Wednesday,Sunny thenSlight ChanceT-storms,High: 86 °F,Wednesday: A 20 percent chance of showers and ...
7,WednesdayNight,Slight ChanceT-storms,Low: 64 °F,Wednesday Night: A 20 percent chance of shower...
8,Thursday,Mostly Sunnythen SlightChanceT-storms,High: 80 °F,Thursday: A 20 percent chance of showers and t...


In [14]:
# Write to excel

writer = pd.ExcelWriter('file_name.xlsx', engine='xlsxwriter')
weather.to_excel(writer)
writer.save()

In [15]:
# Analysis

temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    79
1    65
2    83
3    66
4    83
5    65
6    86
7    64
8    80
Name: temp_num, dtype: object

In [16]:
weather["temp_num"].mean()

74.55555555555556